In [37]:
import tkinter as tk
from tkinter import ttk, messagebox
import joblib
import numpy as np

# Load the CatBoost models for each combination
model_paths = {
    "C1": "C:/Users/asus1/Desktop/CGB1.joblib",
    "C2": "C:/Users/asus1/Desktop/CGB2.joblib",
    "C3": "C:/Users/asus1/Desktop/CGB3.joblib",
    "C4": "C:/Users/asus1/Desktop/CGB4.joblib"
}

models = {}
for key, path in model_paths.items():
    try:
        models[key] = joblib.load(path)
    except FileNotFoundError:
        messagebox.showerror("Model Error", f"Model file for {key} not found. Please ensure the path is correct.")
        exit()

def predict():
    try:
        # Collect the input values based on selected checkboxes
        inputs = []
        selected_vars = [var for var in input_vars.values() if var.get()]
        selected_entries = [entry for var, entry in zip(input_vars.values(), entry_fields.values()) if var.get()]

        if not selected_entries:
            messagebox.showerror("Input Error", "Please select at least one input parameter.")
            return

        for entry in selected_entries:
            value = entry.get()
            if not value.replace('.', '', 1).isdigit() or float(value) < 0:
                messagebox.showerror("Input Error", "Please enter valid non-negative numbers for all selected inputs.")
                return
            # Convert decimal input to whole number (round to nearest integer)
            inputs.append(round(float(value)))

        # Select model according to input variable selection
        model = models["C1"]  # Default model; adjust based on selection logic
        prediction = model.predict([inputs])[0]
        # Convert prediction to a whole number
        output_label.config(text=f"{round(prediction)}")

    except Exception as e:
        messagebox.showerror("Prediction Error", f"An error occurred: {e}")

def toggle_entry(state, entry):
    entry.config(state="normal" if state else "disabled")
    if not state:
        entry.delete(0, tk.END)

def clear_all():
    # Clear all selections and inputs
    for var in input_vars.values():
        var.set(False)
    for entry in entry_fields.values():
        entry.config(state="disabled")
        entry.delete(0, tk.END)
    output_label.config(text="")

# Create main window
root = tk.Tk()
root.title("Ag-MAR Prediction Using Machine Learning Models")
root.geometry("800x500")
root.configure(bg="#E9F1F7")  # Lighter background for better contrast

# Create a canvas widget for layout control
canvas = tk.Canvas(root, width=800, height=650, bg="#E9F1F7")
canvas.pack()

# Function to create a label on the canvas
def create_label(text, font, fg_color, bg_color, x, y):
    label = tk.Label(root, text=text, font=font, fg=fg_color, bg=bg_color)
    canvas.create_window(x, y, window=label)
    return label

# Title and header
title_label = create_label("Ag-MAR Prediction Using Machine Learning Models", 
                           ("Arial", 18, "bold"), "#003366", "#E9F1F7", 400, 30)

developers_text = "Developers: Mohamed Kamel Elshaarawy, Khaled M. Bali & Mohamed Galal Eltarabily"
developers_label = create_label(developers_text, ("Arial", 12), "#333333", "#E9F1F7", 400, 60)

# Column headers for Input, Value, and Include?
create_label("Input", ("Arial", 12, "bold"), "#000000", "#FFFFFF", 150, 100)
create_label("Value", ("Arial", 12, "bold"), "#000000", "#FFFFFF", 450, 100)
create_label("Include?", ("Arial", 12, "bold"), "#000000", "#FFFFFF", 650, 100)

# Input descriptions
input_descriptions = [
    "It (Applied winter water, mm)",
    "Pt (Precipitation, mm)",
    "ETa (Actual evapotranspiration, mm)",
    "ΔS (Change in soil storage, mm)",
    "%O2 (Oxygen level)"
]

# Input variables and initial state
input_vars = {
    "It": tk.BooleanVar(value=True),
    "Pt": tk.BooleanVar(value=True),
    "ETa": tk.BooleanVar(value=True),
    "DeltaSt": tk.BooleanVar(value=False),
    "O2": tk.BooleanVar(value=False)
}

# Entry fields dictionary
entry_fields = {
    "It": tk.Entry(root, font=("Arial", 12, 'bold'), width=20),
    "Pt": tk.Entry(root, font=("Arial", 12, 'bold'), width=20),
    "ETa": tk.Entry(root, font=("Arial", 12, 'bold'), width=20),
    "DeltaSt": tk.Entry(root, font=("Arial", 12, 'bold'), width=20),
    "O2": tk.Entry(root, font=("Arial", 12, 'bold'), width=20)
}

# Disable DeltaSt and O2 initially
entry_fields["DeltaSt"].config(state="disabled")
entry_fields["O2"].config(state="disabled")

# Create labels and entry fields dynamically
entry_y_position = 140
checkbox_y_position = 130
for idx, key in enumerate(entry_fields.keys()):
    label = create_label(input_descriptions[idx], ("Arial", 12, 'bold', 'italic'), "#000000", "#FFFFFF", 150, entry_y_position)
    canvas.create_window(450, entry_y_position + 20, window=entry_fields[key])

    checkbox = tk.Checkbutton(root, variable=input_vars[key], bg="white", command=lambda k=key: toggle_entry(input_vars[k].get(), entry_fields[k]))
    canvas.create_window(650, checkbox_y_position + 20, window=checkbox)

    entry_y_position += 40
    checkbox_y_position += 40

# Output frame for prediction
output_frame = ttk.LabelFrame(root, text="Prediction Result (Recharge (mm)", padding=(30,15), width=350)
output_frame.place(x=150, y=350)

output_label = ttk.Label(output_frame, text="", font=("Arial", 16, 'bold'), foreground="red")
output_label.grid(row=0, column=1, sticky="nsew", padx=10, pady=5)

# Buttons frame (Placed beside the Prediction Result box)
buttons_frame = tk.Frame(root, bg="#E9F1F7")
buttons_frame.place(x=520, y=350)

# Predict button
predict_button = tk.Button(buttons_frame, text="Predict", font=("Arial", 12), command=predict, fg='#FFFFFF', bg='green', width=18)
predict_button.pack(pady=10)

# Clear button
clear_button = tk.Button(buttons_frame, text="Clear", font=("Arial", 12), command=clear_all, fg='#FFFFFF', bg='blue', width=18)
clear_button.pack(pady=10)

# Start the main loop
root.mainloop()
